In [1]:
import os
import numpy as np
import pandas as pd
import pynndescent
import duckdb
import pickle
import scipy.sparse
import importlib
import json

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [2]:
HASHED_FEATURES        = os.environ["HASHED_FEATURES"]
HASHED_FEATURES_IDX    = os.environ["HASHED_FEATURES_IDX"]
KNN_RANDOM_STATE       = int(os.environ["KNN_RANDOM_STATE"])
KNN_METRIC             = os.environ["KNN_METRIC"]
KNN_METRIC_PARAMS      = json.loads(os.environ["KNN_METRIC_PARAMS"]) if json.loads(os.environ["KNN_METRIC_PARAMS"]) else None
KNN_NEIGHBORS          = int(os.environ["KNN_NEIGHBORS"])
KNN_INDEX              = os.environ["KNN_INDEX"]

print(f"""
HASHED_FEATURES        = {HASHED_FEATURES}
HASHED_FEATURES_IDX    = {HASHED_FEATURES_IDX}
KNN_RANDOM_STATE       = {KNN_RANDOM_STATE}
KNN_METRIC             = {KNN_METRIC}
KNN_METRIC_PARAMS      = {KNN_METRIC_PARAMS}
KNN_NEIGHBORS          = {KNN_NEIGHBORS}
KNN_INDEX              = {KNN_INDEX}
""")


HASHED_FEATURES        = ./DATA/EXPERIMENTO-01/hashed_features.dat
HASHED_FEATURES_IDX    = ./DATA/EXPERIMENTO-01/hashed_features_idx.parquet
KNN_RANDOM_STATE       = 42
KNN_METRIC             = manhattan
KNN_NEIGHBORS          = 250
KNN_INDEX              = ./DATA/EXPERIMENTO-01/knn_index.pickle



Os dados gerados pelo notebook anterior são lidos abaixo

In [3]:
hashed_features = scipy.sparse.load_npz(HASHED_FEATURES + ".npz").tocsr()
hashed_features.shape

(91798, 100000)

In [4]:
type(hashed_features)

scipy.sparse._csr.csr_matrix

In [5]:
hashed_features_idx = pd.read_parquet(HASHED_FEATURES_IDX)
hashed_features_idx.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91798 entries, 0 to 91797
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          91798 non-null  int64 
 1   chave_usuario  91798 non-null  object
 2   lotacao_topo   91798 non-null  object
 3   sigla_lotacao  91798 non-null  object
 4   tipo_usuario   91798 non-null  object
 5   cargo          91798 non-null  object
 6   enfase         91798 non-null  object
 7   funcao         91798 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.6+ MB


A biblioteca PyNNDescent é utilizada na classe abaixo para indexar a matriz esparsa de usuários x hashed features.

Como esta funcionalidade precisará ser reutilizada no recomendador, estamos escrevendo a mesma em uma classe separada para possibilitar posterior reuso.

In [6]:
%%writefile knn_indexer.py
import numpy as np
import pickle
import pynndescent

class KnnIndex:
    
    def __init__(self, metric, metric_params, n_neighbors, random_state):
        self.metric        = metric
        self.metric_params = metric_params
        self.n_neighbors   = n_neighbors
        self.random_state  = random_state
        self.data          = None
        self.neighbors_of  = None
        self.distances_of  = None

    def index(self, data):
        knn_index = pynndescent.NNDescent(
            data         = data
        ,   metric       = self.metric
        ,   metric_kwds  = self.metric_params 
        ,   n_neighbors  = self.n_neighbors
        ,   random_state = self.random_state
        ,   n_jobs       = -1
        ,   low_memory   = False
        ,   verbose      = True
        ,   compressed   = False # neighbor_graph is deleted when the index is compressed
        )
        neighbors, distances = knn_index.neighbor_graph
        self.data            = data
        self.neighbors_of    = neighbors.astype("int32", copy=False)
        self.distances_of    = distances.astype("float32", copy=False)

    def save(self, filename):
        with open(filename, "wb") as fh:
            pickle.dump(self, fh)
            
    def get_row(self, idx):
        assert self.data is not None
        return self.data[idx]
    
    def get_neighbors(self, idx):
        assert self.data is not None
        neighbors = self.neighbors_of[idx]
        distances = self.distances_of[idx]
        return neighbors, distances
    
    @classmethod
    def load(klass, filename):
        with open(filename, "rb") as fh:
            return pickle.load(fh)

Overwriting knn_indexer.py


O código abaixo serve para recarregar o módulo em que a classe **KnnIndex** é criada.

Por padrão um módulo não é reimportado caso uma instrução import seja repetida

In [7]:
import knn_indexer
importlib.reload(knn_indexer)
from knn_indexer import KnnIndex

O método abaixo indexa os 85.000+ usuários indicando quais são os KNN_NEIGHBORS mais próximos utilizando a biblioteca PyNNDescent.

Esta é a parte mais lenta do processamento da solução, apesar do uso de múltiplas CPU's.

Dado a elevada dimensionalidade do conjunto de dados somada ao uso de KNN_NEIGHBORS(atualmente 250), a biblioteca PyNNDescent não consegue criar o grafo de vizinhos que possibilita o cômputo dinâmico do KNN aproximado (grafo criado implicitamente na primeira query ou quando o método *prepare* é chamado). Ao tentar criar o grafo, o consumo de memória excede 20+ gigas e ocasiona no término do processo.

Esta limitação indica que o recomendador não funciona para usuários não presentes no conjunto de dados processado.

Para que seja possível recomendar perfis para usuários ainda não vistos, seria preciso avaliar algumas das possibilidades abaixo:

 1 - Entender a causa da explosão do consumo de memória dentro da biblioteca PyNNDescent. Potencialmente sendo causada pela  convertão (possivelmente implícita) de linhas da matriz esparsa (que hoje utiliza 500.000 colunas) em linhas densas.
 
 2 - Utilizar alguma técnica de redução de dimensionalidade, como por exemplo o refatoramento da matriz usando **Singular Value Decomposition** ou **Non-Negative Matrix Factorization**.
 
 3 - Reduzir o conjunto de dados global por múltiplos conjuntos de dados por lotação topo.
 
 Estes 3 pontos, entretanto, encontram-se fora do escopo desse trabalho

In [8]:
knn_index = KnnIndex(
    metric        = KNN_METRIC
,   metric_params = KNN_METRIC_PARAMS
,   n_neighbors   = KNN_NEIGHBORS
,   random_state  = KNN_RANDOM_STATE
)
knn_index.index(hashed_features)

Sun Aug 27 17:35:18 2023 Building RP forest with 22 trees
Sun Aug 27 17:35:37 2023 metric NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
	 5  /  16
	 6  /  16
	 7  /  16
	Stopping threshold met -- exiting after 7 iterations


Uma vez calculado os K vizinhos mais próximos, é possível obter a lista de índices dos vizinhos assim como suas respectivas distâncias. O arquivo parquet HASHED_FEATURES_IDX contém os índices das linhas correspondentes a cada usuário no conjunto de dados processado.

Atualmente a métrica de distância utilizada é a distância do cosseno baseada na similaridade do cosseno (1 - similaridade). Esta distância calcula basicamente o ângulo em espaço n-vetorial que dois vetores formam entre si, sendo que quanto menor a distância mais próximos estes vetores se encontram. A distância varia entre 0.0 e 1.0 e este intervalo limitado é utilizado posteriormente na fórmula de *scoring* dos perfis recomendados

In [9]:
neighbors, distances = knn_index.get_neighbors(0)
display(neighbors)
display(distances)

array([    0, 29301, 16115, 85550, 27715, 69840, 29939, 29008, 25392,
       19955, 40571, 61978,   958,  8853, 27035, 55804, 11492, 26039,
        5356, 63071, 61360,  5624, 62911, 10523, 27483,  5663, 20132,
       27484, 30088,  3673, 25557, 68230, 14074, 63010, 57121, 26533,
       26093, 14721, 62846, 28929, 26792, 35310, 70013,  1609, 34924,
       31611, 85531, 26134, 76791, 26430, 58558, 28586, 64330, 34070,
       35295,  5596, 87270,  9960, 30490, 35230, 34123, 28583,  8680,
       33236,  1021,  5862, 35024, 14868, 63679, 28390, 64022, 19596,
       34034, 68663,  1207, 62974, 18846, 34923,  1707, 25323, 35710,
       24243, 58823, 40446,  2222, 26534, 30285,  2241, 27355,  2235,
       26416, 13147, 26990, 85494, 58697, 40488, 63805, 18435, 73850,
       11969, 32633,  2229, 87377, 26193, 26927, 39901, 76662, 40579,
        2923, 70699,  3185, 16682, 18450,  2937, 32675, 26857, 90735,
       32641, 78922, 31347, 32624, 70475, 28617,  2223, 26635, 16132,
         155, 73762,

array([ 0.       ,  9.8494625, 10.169329 , 12.091906 , 12.121661 ,
       12.275476 , 13.914285 , 13.970804 , 13.979214 , 13.983862 ,
       14.044023 , 14.10435  , 14.469419 , 14.5154915, 14.598303 ,
       14.896113 , 14.943519 , 14.968422 , 15.047182 , 15.07796  ,
       15.117229 , 15.139833 , 15.157316 , 15.194816 , 15.358604 ,
       15.490989 , 15.607591 , 15.632371 , 15.664402 , 15.872202 ,
       15.929386 , 15.973163 , 16.023907 , 16.085327 , 16.200129 ,
       16.282007 , 16.297909 , 16.351389 , 16.668652 , 16.89818  ,
       17.00733  , 17.108398 , 17.221188 , 17.44167  , 17.64549  ,
       17.6684   , 18.03335  , 18.03909  , 18.202145 , 18.386244 ,
       19.075531 , 19.147734 , 19.227869 , 19.371809 , 19.491138 ,
       19.638369 , 19.76654  , 19.813269 , 19.869047 , 19.881018 ,
       20.264196 , 20.330889 , 20.413258 , 20.807522 , 20.811481 ,
       21.049875 , 21.277863 , 21.58285  , 21.846647 , 22.107939 ,
       22.431646 , 22.563532 , 22.574785 , 23.868124 , 23.8776

O índice, uma vez calculado, é salvo para poder ser consultado posteriormente.

In [10]:
knn_index.save(KNN_INDEX)
print(f"knn index has been built -> {KNN_INDEX}")

knn index has been built -> ./DATA/EXPERIMENTO-01/knn_index.pickle
